In [ ]:
df_ticker = pd.read_csv('/content/drive/MyDrive/코스피200ticker.csv', dtype={'ticker':str})
df_ticker

,ticker,start_date,end_date
0,000030,2015-01-30,2019-01-31
1,000050,2015-06-30,2018-05-31
2,000060,2022-06-30,2022-12-29
3,000070,2015-01-30,2022-11-30
4,000080,2015-01-30,2024-12-30
...,...,...,...
318,450080,2023-12-28,2024-12-30
319,454910,2024-06-28,2024-12-30
320,456040,2024-12-30,2024-12-30
321,457190,2024-06-28,2024-12-30


In [ ]:
import pandas as pd
import numpy as np

# Load the original data
df = pd.read_csv('/content/drive/MyDrive/뉴스분류/CJ_predicted_results_hope.csv', dtype={'ticker':str})
df['date'] = pd.to_datetime(df['일자'], format='%Y%m%d')
df
# 1. Aggregate by ticker and date
agg = df.groupby(['ticker', 'date']).apply(lambda x: pd.Series({
    'P': (x['pred_label'] == 1).sum(),
    'N': (x['pred_label'] == -1).sum(),
    'Z': (x['pred_label'] == 0).sum()
})).reset_index()

# 2. Create a continuous date range for each ticker and reindex
def fill_missing_dates(group):
    ticker = group['ticker'].iloc[0]
    # min_date = group['date'].min()
    # max_date = group['date'].max()
    df_ticker = pd.read_csv('/content/drive/MyDrive/코스피200ticker.csv', dtype={'ticker':str})
    start_date = df_ticker[df_ticker['ticker'] == ticker]['start_date'].values[0]
    start_date = (pd.to_datetime(start_date) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
    end_date = df_ticker[df_ticker['ticker'] == ticker]['end_date'].values[0]
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

    group = group.set_index('date').reindex(all_dates).reset_index()
    group['date'] = group['index']
    group['ticker'] = ticker
    group[['P', 'N', 'Z']] = group[['P', 'N', 'Z']].fillna(0)
    return group.drop(columns=['index'])

agg_continuous = agg.groupby('ticker', group_keys=False).apply(fill_missing_dates)
agg_continuous['T'] = agg_continuous['P'] + agg_continuous['N'] + agg_continuous['Z']

# 3. Calculate features on the continuous timeline
k = 5
agg_continuous['news_sentiment'] = (agg_continuous['P'] - agg_continuous['N']) / (agg_continuous['T'] + k)
agg_continuous['news_conviction'] = (agg_continuous['P'] - agg_continuous['N']).abs() / (agg_continuous['T'] + k)
agg_continuous['news_volume'] = np.log1p(agg_continuous['T'])

# 4. Calculate EWM on the continuous timeline
agg_continuous['news_sentiment_ewm7'] = agg_continuous.groupby('ticker')['news_sentiment'].transform(lambda x: x.ewm(span=7, adjust=False).mean())
agg_continuous['news_sentiment_ewm20'] = agg_continuous.groupby('ticker')['news_sentiment'].transform(lambda x: x.ewm(span=20, adjust=False).mean())
agg_continuous['news_sentiment_surprise'] = agg_continuous['news_sentiment'] - agg_continuous['news_sentiment_ewm20']

# 5. Apply Lag 1 Policy (Shift all features by 1 day)
# feature_cols = ['news_sentiment', 'news_conviction', 'news_volume', 'news_sentiment_ewm5', 'news_sentiment_ewm20', 'news_sentiment_surprise']
# agg_lagged = agg_continuous.copy()
# for col in feature_cols:
#     agg_lagged[col] = agg_continuous.groupby('ticker')[col].shift(1)

# # Inspect a specific period (e.g., first 10 days) to show the effect
# print(agg_lagged.head(15))

# Filter to show only dates where there was news originally, or keep all?
# Usually, dataset_daily has all trading days. Keeping all dates is better.
# agg_lagged.to_csv('news_sentiment_features_continuous_lag1.csv', index=False)

/tmp/ipython-input-2130227808.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg = df.groupby(['ticker', 'date']).apply(lambda x: pd.Series({


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import pandas as pd
import numpy as np

# Load the original data
df = pd.read_csv('/content/drive/MyDrive/뉴스분류/CJ_predicted_results_hope.csv', dtype={'ticker':str})
df['date'] = pd.to_datetime(df['일자'], format='%Y%m%d')

# 1. Aggregate by ticker and date
agg = df.groupby(['ticker', 'date']).apply(lambda x: pd.Series({
    'P': (x['pred_label'] == 1).sum(),
    'N': (x['pred_label'] == -1).sum(),
    'Z': (x['pred_label'] == 0).sum()
})).reset_index()

# 2. Create a continuous date range for each ticker and reindex
def fill_missing_dates(group):
    ticker = group['ticker'].iloc[0]
    # min_date = group['date'].min()
    # max_date = group['date'].max()
    all_dates = pd.date_range(start='2015-01-01', end='2024-12-31', freq='D')

    group = group.set_index('date').reindex(all_dates).reset_index()
    group['date'] = group['index']
    group['ticker'] = ticker
    group[['P', 'N', 'Z']] = group[['P', 'N', 'Z']].fillna(0)
    return group.drop(columns=['index'])

agg_continuous = agg.groupby('ticker', group_keys=False).apply(fill_missing_dates)
agg_continuous['T'] = agg_continuous['P'] + agg_continuous['N'] + agg_continuous['Z']

# 3. Calculate features on the continuous timeline
k = 5
agg_continuous['news_sentiment'] = (agg_continuous['P'] - agg_continuous['N']) / (agg_continuous['T'] + k)
agg_continuous['news_conviction'] = (agg_continuous['P'] - agg_continuous['N']).abs() / (agg_continuous['T'] + k)
agg_continuous['news_volume'] = np.log1p(agg_continuous['T'])

# 4. Calculate EWM on the continuous timeline
agg_continuous['news_sentiment_ewm7'] = agg_continuous.groupby('ticker')['news_sentiment'].transform(lambda x: x.ewm(span=7, adjust=False).mean())
agg_continuous['news_sentiment_ewm30'] = agg_continuous.groupby('ticker')['news_sentiment'].transform(lambda x: x.ewm(span=30, adjust=False).mean())
agg_continuous['news_sentiment_surprise'] = agg_continuous['news_sentiment'] - agg_continuous['news_sentiment_ewm20']

# # 5. Apply Lag 1 Policy (Shift all features by 1 day)
# feature_cols = ['news_sentiment', 'news_conviction', 'news_volume', 'news_sentiment_ewm5', 'news_sentiment_ewm20', 'news_sentiment_surprise']
# agg_lagged = agg_continuous.copy()
# for col in feature_cols:
#     agg_lagged[col] = agg_continuous.groupby('ticker')[col].shift(1)

# # Inspect a specific period (e.g., first 10 days) to show the effect
# print(agg_lagged.head(15))

# # Filter to show only dates where there was news originally, or keep all?
# # Usually, dataset_daily has all trading days. Keeping all dates is better.
# agg_lagged.to_csv('news_sentiment_features_continuous_lag1.csv', index=False)

/tmp/ipython-input-4008213553.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg = df.groupby(['ticker', 'date']).apply(lambda x: pd.Series({
/tmp/ipython-input-4008213553.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_continuous = agg.groupby('ticker', group_keys=False).apply(fill_missing_dates)


In [ ]:
agg_continuous.to_csv('/content/drive/MyDrive/CJ_뉴스점수6개.csv', index=False, encoding="utf-8-sig")

In [ ]:
import os
os.listdir('/content/drive/MyDrive/뉴스분류/')

['CJ_predicted_results_hope.csv',
 'DB손해보험_predicted_results_hope.csv',
 'BGF리테일_predicted_results_hope.csv',
 'DL이앤씨_predicted_results_hope.csv',
 'DGB금융지주_predicted_results_hope.csv',
 'DL_predicted_results_hope.csv',
 'CJ대한통운_predicted_results_hope.csv',
 'BNK금융지주_predicted_results_hope.csv',
 'GS_predicted_results_hope.csv',
 'GS건설_predicted_results_hope.csv',
 'HD한국조선해양_predicted_results_hope.csv',
 'GS리테일_predicted_results_hope.csv',
 'HD현대미포_predicted_results_hope.csv',
 'HD현대_predicted_results_hope.csv',
 'HD현대인프라코어_predicted_results_hope.csv',
 'F&F_predicted_results_hope.csv',
 'GKL_predicted_results_hope.csv',
 'HD현대중공업_predicted_results_hope.csv',
 'HMM_predicted_results_hope.csv',
 'HL만도_predicted_results_hope.csv',
 'HS효성첨단소재_predicted_results_hope.csv',
 'HD현대일렉트릭_predicted_results_hope.csv',
 'JB금융지주_predicted_results_hope.csv',
 'KCC_predicted_results_hope.csv',
 'K

In [ ]:
len(os.listdir('/content/drive/MyDrive/뉴스분류/'))

200

In [ ]:
# Load the original data
df = pd.read_csv('/content/drive/MyDrive/뉴스분류/CJ_predicted_results_hope.csv', dtype={'ticker':str})
df['date'] = pd.to_datetime(df['일자'], format='%Y%m%d')

# 1. Aggregate by ticker and date
agg = df.groupby(['ticker', 'date']).apply(lambda x: pd.Series({
    'P': (x['pred_label'] == 1).sum(),
    'N': (x['pred_label'] == -1).sum(),
    'Z': (x['pred_label'] == 0).sum()
})).reset_index()

# 2. Create a continuous date range for each ticker and reindex
def fill_missing_dates(group):
    ticker = group['ticker'].iloc[0]
    # min_date = group['date'].min()
    # max_date = group['date'].max()
    all_dates = pd.date_range(start='2015-01-01', end='2024-12-31', freq='D')

    group = group.set_index('date').reindex(all_dates).reset_index()
    group['date'] = group['index']
    group['ticker'] = ticker
    group[['P', 'N', 'Z']] = group[['P', 'N', 'Z']].fillna(0)
    return group.drop(columns=['index'])

agg_continuous = agg.groupby('ticker', group_keys=False).apply(fill_missing_dates)
agg_continuous['T'] = agg_continuous['P'] + agg_continuous['N'] + agg_continuous['Z']

# 3. Calculate features on the continuous timeline
k = 5
agg_continuous['news_sentiment'] = (agg_continuous['P'] - agg_continuous['N']) / (agg_continuous['T'] + k)
agg_continuous['news_conviction'] = (agg_continuous['P'] - agg_continuous['N']).abs() / (agg_continuous['T'] + k)
agg_continuous['news_volume'] = np.log1p(agg_continuous['T'])

# 4. Calculate EWM on the continuous timeline
agg_continuous['news_sentiment_ewm7'] = agg_continuous.groupby('ticker')['news_sentiment'].transform(lambda x: x.ewm(span=7, adjust=False).mean())
agg_continuous['news_sentiment_ewm30'] = agg_continuous.groupby('ticker')['news_sentiment'].transform(lambda x: x.ewm(span=30, adjust=False).mean())
agg_continuous['news_sentiment_surprise'] = agg_continuous['news_sentiment'] - agg_continuous['news_sentiment_ewm20']

In [ ]:
import pandas as pd

In [ ]:

def simple_news_sentiment(df):
# 1. 데이터 로드
  df['date'] = pd.to_datetime(df['일자'], format='%Y%m%d')

  # 2. 종목별/날짜별 집계
  # P: 긍정(+1) 기사 수, N: 부정(-1) 기사 수, T: 전체 기사 수
  agg = df.groupby(['ticker', 'date']).apply(lambda x: pd.Series({
      'P': (x['pred_label'] == 1).sum(),
      'N': (x['pred_label'] == -1).sum(),
      'T': len(x)
  })).reset_index()

  # 3. news_sentiment 계산 (보정상수 k=5)
  # 수식: (긍정수 - 부정수) / (전체기사수 + k)
  k = 5
  agg['news_sentiment'] = (agg['P'] - agg['N']) / (agg['T'] + k)

  # 필요한 컬럼만 추출
  result = agg[['ticker', 'date', 'news_sentiment']]

  return result

In [ ]:
company_list = os.listdir('/content/drive/MyDrive/뉴스분류')

In [ ]:
for company in company_list:
  df = pd.read_csv(f'/content/drive/MyDrive/뉴스분류/{company}', dtype={'ticker':str})
  df['ticker'] = df['ticker'].str.zfill(6)
  ticker = df['ticker'][0]
  date_index = pd.date_range(start='2015-01-01', end='2024-12-31', freq='D')
  score = simple_news_sentiment(df)
  df_dates = pd.DataFrame(date_index, columns=['date'])
  merge_df = pd.merge(df_dates, score, on='date', how='left')
  merge_df['ticker'] = ticker
  merge_df.fillna(0, inplace=True)
